In [6]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Custom Dataset Class
class MNISTDigitsDataset(Dataset):
    def __init__(self, root_dirs, transform=None):
        #self.image_dir = image_dir
        #self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
        #self.transform = transform

        self.image_paths = []
        for root_dir in root_dirs:
            for file in os.listdir(root_dir):
                if file.endswith('.png'):
                    self.image_paths.append(os.path.join(root_dir, file))
        self.transform = transform


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        #image_path = os.path.join(self.image_dir, image_name)
        


        # Load the image
        image = Image.open(image_path).convert('L')  # Convert to grayscale

        image_name = os.path.basename(image_path)
        # Extract label from the filename (e.g., '0428.png' -> [0, 4, 2, 8])
        label = [int(digit) for digit in image_name.split('.')[0]]
        

        #added
        label_num = torch.sum(torch.tensor(label)).item()  # Sum the elements and convert to a Python scalar


        # Convert label to one-hot encoding (4 digits, 10 classes per digit)
        one_hot_label = torch.zeros(40, dtype=torch.long)  # 10 classes * 4 digits = 40
        for i, digit in enumerate(label):
            one_hot_label[i * 10 + digit] = 1  # Set the corresponding class to 1
        
        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)
        
        return image, one_hot_label ,label_num ,image_name

# Define image transformations
transform = transforms.Compose([
#    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),  # Random rotation ±15° and shifts up to 10%
    transforms.Resize((40, 168)),  # Resize image to the correct size
    transforms.ToTensor(),         # Convert image to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize (for grayscale images)
])

# Create Dataset and DataLoader
#image_dir = r"./exterim/images"  # Update with your path
root_dirs = [
    r"./exterim/images",  # Update with your paths
    r"./exterim/images2"  # Add additional directories here
]

dataset = MNISTDigitsDataset(root_dirs=root_dirs, transform=transform)

from torch.utils.data import random_split
print("length of dataset",len(dataset))
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size


# DataLoader for batching
batch_size = 16

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader for training and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



length of dataset 2775


In [7]:

#class 1


import torch
import torch.nn as nn
import torch.nn.functional as F

class MNISTDigitModel(nn.Module):
    def __init__(self, num_blocks, kernel_size, activation, pool, dropout):
        super(MNISTDigitModel, self).__init__()
        self.num_blocks = num_blocks
        self.kernel_size = kernel_size
        self.activation = activation
        self.pool = pool
        self.dropout = dropout
        
        layers = []
        in_channels = 1  # Grayscale input images
        out_channels = 64  # Initial number of filters
        
        # Add convolutional blocks
        for _ in range(num_blocks):
            layers.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding='same'),
                self._get_activation(activation),
                nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding='same'),
                self._get_activation(activation),
                self._get_pool(pool),
                nn.Dropout(dropout)
            ))
            in_channels = out_channels
            out_channels *= 2  # Double the filters after each block
            
        
        self.conv_blocks = nn.Sequential(*layers)
        
        # Dummy input to calculate the flattened size
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, 40, 168)
            flattened_size = self.conv_blocks(dummy_input).numel()
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flattened_size, 512),
            self._get_activation(activation),
            nn.Dropout(dropout),
            nn.Linear(512, 40)  # 40 output classes (10 per digit for 4 digits)
        )
    
    def _get_activation(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'sigmoid':
            return nn.Sigmoid()
        else:
            raise ValueError("Activation not supported")
    
    def _get_pool(self, pool):
        if pool == 'max':
            return nn.MaxPool2d(2)
        elif pool == 'avg':
            return nn.AvgPool2d(2)
        else:
            raise ValueError("Pooling method not supported")
    
    def forward(self, x):
        x = self.conv_blocks(x)
        x = self.fc(x)

        x = x.view(-1, 4, 10)
        return x


In [8]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class MNISTSumModel(nn.Module):
    def __init__(self):
        super(MNISTSumModel, self).__init__()
        
        # MLP layers (flatten the 1x4x10 output)
        self.fc1 = nn.Linear(40, 64)  # 40 input features (4 digits * 10 classes)
        self.fc22 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Output a single scalar (the sum of digits)

    def forward(self, x):
        # Apply softmax to each 10-length vector (per digit)
        x = x.float()
        x = F.softmax(x, dim=-1)
        
        # Flatten the input (4 digits * 10 classes)
        x = x.view(-1, 40)  # Shape becomes (batch_size, 40)

        # MLP layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc22(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Single scalar output
        return x
    


In [9]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# Instantiate the models
new_dropout = 0.1
digit_model = MNISTDigitModel(num_blocks=5, kernel_size=3, activation='relu', pool='max', dropout=new_dropout)
sum_model = MNISTSumModel()



In [10]:

# checkpoint_dir = './checkpoints'
# os.makedirs(checkpoint_dir, exist_ok=True)
# latest_checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint_epoch_621.pth')
# start_epoch = 0
# ## not needed as we have combined now
# if os.path.exists(latest_checkpoint_path):
#     print(f"Loading checkpoint from {latest_checkpoint_path}...")
#     checkpoint = torch.load(latest_checkpoint_path)
#     digit_model.load_state_dict(checkpoint['model_state_dict'])
#     start_epoch = checkpoint['epoch']
#     print(f"Resuming training from epoch {start_epoch}...")

#     for param in digit_model.parameters():
#         param.requires_grad = False

# ## not needed as we have combined now
# sum_model_path = "checkpoints/decoder.pth"
# if os.path.exists(sum_model_path):
#     print(f"Loading pretrained MNISTSumModel from {sum_model_path}...")
#     sum_model.load_state_dict(torch.load(sum_model_path))

#     # for param in sum_model.parameters():
#     #     param.requires_grad = False


for param in digit_model.parameters():
    param.requires_grad = False


In [11]:

# Define combined model
class CombinedModel(nn.Module):
    def __init__(self, digit_model, sum_model):
        super(CombinedModel, self).__init__()
        self.digit_model = digit_model
        self.sum_model = sum_model

    def forward(self, x):
        digit_output_encoded = self.digit_model(x)  # Shape: [batch_size, 4, 10]
        sum_output = self.sum_model(digit_output_encoded)  # Shape: [batch_size, 1]
        return digit_output_encoded, sum_output

combined_model = CombinedModel(digit_model, sum_model)



# Optimizer and criterion
new_learning_rate = 0.0001 ##0.0001 for CNN
optimizer = optim.Adam(filter(lambda p: p.requires_grad, combined_model.parameters()), lr=new_learning_rate)
# optimizer = optim.Adam([
#     {'params': filter(lambda p: p.requires_grad, digit_model.parameters()), 'lr': 0.00001},
# ])

# optimizer = optim.Adam([
#     {'params': digit_model.parameters(), 'lr': 0.00001},  # Learning rate for digit_model
#     {'params': sum_model.parameters(), 'lr': 0.0},  # Frozen sum_model (effectively ignored)
# ])

# Load the combined model checkpoint
checkpoint_path = './checkpoints_comb/checkpoint_epoch_700.pth'
start_epoch = 0
if os.path.exists(checkpoint_path):
    print(f"Loading combined model checkpoint from {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path)
    
    # Load the combined model weights
    combined_model.load_state_dict(checkpoint['model_state_dict'])
    # Set the starting epoch for resuming training
    start_epoch = checkpoint['epoch']
    print(f"Resuming training from epoch {start_epoch}...")

# criterion_digit = nn.CrossEntropyLoss()  # For digit classification
criterion_sum = nn.MSELoss()  # For sum regression (used only for evaluation)


# Training loop for combined model
num_epochs = 10000
checkpoint_dir = './checkpoints_comb'  # Directory for saving checkpoints
os.makedirs(checkpoint_dir, exist_ok=True)  # Ensure the checkpoint directory exists

for epoch in range(start_epoch, start_epoch + num_epochs):
    combined_model.train()  # Set model to training mode
    # running_loss_digit = 0.0
    # running_loss_sum = 0.0
    running_loss = 0.0
    for images, _, labels_sum, _ in train_loader:  # Adjust based on your dataset
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        
        _, sum_output = combined_model(images)
        # sum_output = sum_output.view(-1) 
        # Compute loss (compare output to label_num)
        loss = criterion_sum(sum_output.view(-1), labels_sum.float())  # Squeeze to remove extra dimensions
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # Save checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pth')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': combined_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

    # Evaluation on test_loader
    with torch.no_grad():  # Disable gradient computation for evaluation
        for images, _, labels_sum, nam in test_loader:  # Adjust based on your dataset
            # Forward pass
            _, sum_output = combined_model(images)


            # Optionally, compare predicted and actual values
            for i in range(3):
                print(f"Sample :",nam[i])
                print(f"Original Sum: {labels_sum[i]}")
                print(f"Predicted Sum: {sum_output[i].item()}")  # Use .item() to get the scalar value

            break  # Just evaluate the first batch and exit the loop


Loading combined model checkpoint from ./checkpoints_comb/checkpoint_epoch_700.pth...


C:\Users\Pavan\AppData\Local\Temp\ipykernel_13432\2977658306.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Resuming training from epoch 700...
Epoch [701/10000], Loss: 0.2615
Sample : 4418.png
Original Sum: 17
Predicted Sum: 17.041715621948242
Sample : 2308.png
Original Sum: 13
Predicted Sum: 12.943526268005371
Sample : 5567.png
Original Sum: 23
Predicted Sum: 23.0585880279541
Epoch [702/10000], Loss: 0.2345
Sample : 4418.png
Original Sum: 17
Predicted Sum: 17.047210693359375
Sample : 2308.png
Original Sum: 13
Predicted Sum: 12.908239364624023
Sample : 5567.png
Original Sum: 23
Predicted Sum: 22.90321922302246
Epoch [703/10000], Loss: 0.3244
Sample : 4418.png
Original Sum: 17
Predicted Sum: 17.04605484008789
Sample : 2308.png
Original Sum: 13
Predicted Sum: 12.990662574768066
Sample : 5567.png
Original Sum: 23
Predicted Sum: 23.034543991088867
Epoch [704/10000], Loss: 0.1865
Sample : 4418.png
Original Sum: 17
Predicted Sum: 17.21051788330078
Sample : 2308.png
Original Sum: 13
Predicted Sum: 13.010443687438965
Sample : 5567.png
Original Sum: 23
Predicted Sum: 23.127132415771484
Epoch [705/10

KeyboardInterrupt: 

## Evaluation

In [12]:
# Evaluation on test_loader
with torch.no_grad():  # Disable gradient computation for evaluation
    for images, _, labels_sum, nam in test_loader:  # Adjust based on your dataset
        # Forward pass
        _, sum_output = combined_model(images)


        # Optionally, compare predicted and actual values
        for i in range(3):
            print(f"Sample :",nam[i])
            print(f"Original Sum: {labels_sum[i]}")
            print(f"Predicted Sum: {sum_output[i].item()}")  # Use .item() to get the scalar value

        break  # Just evaluate the first batch and exit the loop


Sample : 4418.png
Original Sum: 17
Predicted Sum: 16.9477481842041
Sample : 2308.png
Original Sum: 13
Predicted Sum: 12.901043891906738
Sample : 5567.png
Original Sum: 23
Predicted Sum: 22.8297061920166


#### Even at this stage,its able to predict correct sum, after round off will give correct answer
#### Training it more will reduce MSE